In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pyvis.network import Network
import plotly.graph_objects as go
from scipy.integrate import solve_ivp

In [ ]:
path = "data/networks/"

# Homogeneous graph
G = nx.Graph()
G.add_edges_from(np.loadtxt(path + "homog_regular.txt", dtype=int))
G.number_of_edges(), G.number_of_nodes()

In [ ]:
# Convert node IDs to integers
mapping = {node: i for i, node in enumerate(G.nodes)}
G = nx.relabel_nodes(G, mapping)

In [ ]:
# Convert to Pyvis Network
net = Network(notebook=True)
net.from_nx(G)

# Save to HTML file
net.show("network.html")

In [ ]:
import os

files = os.listdir(path)

for f in files[:-2]:
    name = f.split(".txt")[0]
    G = nx.Graph()
    G.add_edges_from(np.loadtxt(path + f, dtype=int))

    # Convert node IDs to integers
    mapping = {node: i for i, node in enumerate(G.nodes)}
    G = nx.relabel_nodes(G, mapping)

    # Convert to Pyvis Network
    net = Network(notebook=True)
    net.from_nx(G)
    
    # Save to HTML file
    net.show(f"data/visualisations/{name}.html")

In [ ]:
reg = "D:\D_Drive\Github\EPI\Competing_pathogens_Heterogenous_networks\Real_world_data\Oregon_router_networks/nw9_May_26.txt"
G = nx.Graph()
G.add_edges_from(np.loadtxt(reg, dtype=int))

# Convert node IDs to integers
mapping = {node: i for i, node in enumerate(G.nodes)}
G = nx.relabel_nodes(G, mapping)

# Convert to Pyvis Network
net = Network(notebook=True)
net.from_nx(G)

# Save to HTML file
net.show(f"data/visualisations/oregon_router.html")

## ODE simulations

In [2]:
def sisd_ode(t, init, params):
    # Initital fractions
    S, I1, I2, I3, D = init
    # Parameters
    gamma, beta1, beta2, beta3, mu, delta = params

    one_two = mu
    two_three = mu

    # ODEs
    dS = -S*(beta1*I1 + beta2*I2 + beta3*I3) + gamma*(I1 + I2 + I3)
    dI1 = S*beta1*I1 - gamma*I1 - delta*I1 - I1*one_two - beta2*I1*I2 - beta3*I1*I3
    dI2 = S*beta2*I2 - gamma*I2 - delta*I2 - two_three*I2 + one_two*I1 + beta2*I1*I2 - beta3*I2*I3
    dI3 = S*beta3*I3 - gamma*I3 - delta*I3 + two_three*I2 + beta3*I1*I3 + beta3*I2*I3
    dD = delta*(I1 + I2 + I3)

    return [dS, dI1, dI2, dI3, dD]

In [4]:
init = [0.95, 0.05, 0, 0, 0]
params = [0.05, 0.115, 0.12937500000000002, 0.14375000000000002, 5e-5, 1e-4]
T = 2000
times = list(range(T))
runs = 100

In [5]:
%%time
import matplotlib.pyplot as plt

def solve_sys(init, params, times, runs):

    agg = []
    for i in range(runs):
        # Numerically solve the IVP
        sir_sol = solve_ivp(fun=lambda t, y: sisd_ode(t, y, params), t_span=[min(times),max(times)], y0=init, t_eval=times, method="RK45")
        agg.append(sir_sol["y"])
    # Average the results
    agg = np.array(agg)
    print(agg.shape)
    agg = np.mean(agg, axis=0)
    print(agg.shape)

    return agg

agg = solve_sys(init, params, times, runs)

(100, 5, 2000)
(5, 2000)
CPU times: total: 547 ms
Wall time: 598 ms


In [6]:
agg.shape

(5, 2000)

In [7]:
# Create the Plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=agg[0].flatten(), mode='lines+markers',
                    name='Suscetible'))
fig.add_trace(go.Scatter(x=times, y=agg[1].flatten(), mode='lines+markers',
                    name='Variant 1'))
fig.add_trace(go.Scatter(x=times, y=agg[2].flatten(), mode='lines+markers',
                    name='Variant 2'))
fig.add_trace(go.Scatter(x=times, y=agg[3].flatten(), mode='lines+markers',
                    name='Variant 3'))
fig.add_trace(go.Scatter(x=times, y=agg[4].flatten(), mode='lines+markers',
                    name='Deceased'))
fig.update_layout(title='Solution to dy/dt = -ky',
                  xaxis_title='Time',
                  yaxis_title='y')
fig.show()

In [11]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html

# Create the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph'),
    dcc.Slider(
        id='gamma',
        min=0.1,
        max=1.0,
        step=0.1,
        value=params[0],
        marks={i: str(i) for i in np.arange(0.1, 1.1, 0.1)}
    ),
    dcc.Slider(
        id='beta1',
        min=0.1,
        max=1.0,
        step=0.1,
        value=params[1],
        marks={i: str(i) for i in np.arange(0.1, 1.1, 0.1)}
    ),
    dcc.Slider(
        id='beta2',
        min=0.1,
        max=1.0,
        step=0.1,
        value=params[2],
        marks={i: str(i) for i in np.arange(0.1, 1.1, 0.1)}
    ),
    dcc.Slider(
        id='beta3',
        min=0.1,
        max=1.0,
        step=0.1,
        value=params[3],
        marks={i: str(i) for i in np.arange(0.1, 1.1, 0.1)}
    ),
    dcc.Slider(
        id='delta',
        min=0.1,
        max=1.0,
        step=0.1,
        value=params[5],
        marks={i: str(i) for i in np.arange(0.1, 1.1, 0.1)}
    )
])

@app.callback(
    Output('graph', 'figure'),
    Input('gamma', 'value'),
    Input('beta1', 'value'),
    Input('beta2', 'value'),
    Input('beta3', 'value'),
    Input('delta', 'value')
)
def update_graph(k):
    # Solve the ODE with the new parameter
    agg = []
    for i in range(runs):
        # Numerically solve the IVP
        sir_sol = solve_ivp(fun=lambda t, y: sisd_ode(t, y, params), t_span=[min(times),max(times)], y0=init, t_eval=times, method="RK45")
        agg.append(sir_sol["y"])
    # Average the results
    agg = np.array(agg)
    agg = np.mean(agg, axis=0)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t_span, y=sol.flatten(), mode='lines+markers'))
    fig.update_layout(title='Solution to dy/dt = -ky',
                      xaxis_title='Time',
                      yaxis_title='y')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: Dict key must a type serializable with OPT_NON_STR_KEYS

